In [10]:
!pip install -q gradio git+https://github.com/openai/whisper.git groq spotipy requests transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# ✅ STEP 1: Install Required Packages (if needed)
# !pip install -q gradio git+https://github.com/openai/whisper.git groq spotipy requests

# ✅ STEP 2: Import Libraries
import whisper
import gradio as gr
import requests
import os
from dotenv import load_dotenv
from groq import Groq
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Load from .env file
load_dotenv()

# Access environment variables
groq_api_key = os.getenv("GROQ_API_KEY")
client_id = os.getenv("SPOTIPY_CLIENT_ID")
client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")

# If needed, set them explicitly
os.environ["GROQ_API_KEY"] = groq_api_key
os.environ["SPOTIPY_CLIENT_ID"] = client_id
os.environ["SPOTIPY_CLIENT_SECRET"] = client_secret

# ✅ STEP 4: Initialize Clients
client = Groq(api_key=os.environ["GROQ_API_KEY"])
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
asr_model = whisper.load_model("base")

# ✅ STEP 5: Spotify Playlist
def get_playlist(emotion, lang="english", limit=5):
    emotion_to_query = {
        "خوشی": "Pakistani upbeat songs", "اداسی": "Pakistani sad songs",
        "غصہ": "Pakistani rock", "تھکاوٹ": "Pakistani lofi",
        "جذباتی": "Pakistani romantic songs", "پر سکون": "Pakistani sufi music",
        "happy": "feel good", "sad": "melancholy", "angry": "hard rock", "tired": "lofi",
        "exhausted": "ambient", "nostalgic": "throwback", "motivated": "upbeat",
        "demotivated": "healing", "excited": "party", "calm": "chill", "anxious": "relaxing"
    }
    query = emotion_to_query.get(emotion.strip().lower(), "chill")
    results = sp.search(q=query, type='track', limit=limit)
    playlist = []
    for track in results['tracks']['items']:
        name = track['name']
        artist = track['artists'][0]['name']
        url = track['external_urls']['spotify']
        entry = f"🎵 {name} از {artist} → [سنیں]({url})" if lang == "urdu" else f"🎵 {name} by {artist} → [Listen]({url})"
        playlist.append(entry)
    return playlist

# ✅ STEP 6: Generate Poem
def get_poem(emotion, lang):
    prompt = (
        f"ایک دل کو چھو لینے والی اردو نظم لکھیں جو پاکستانی شاعری کے انداز میں ہو۔ جذبہ: {emotion}"
        if lang == "urdu"
        else f"Write a short free-verse poem in {lang} inspired by the emotion: {emotion}. Use deep feeling and vivid imagery."
    )
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# ✅ STEP 7: Mood GIF from Tenor
def search_tenor_gif(mood):
    try:
        url = f"https://g.tenor.com/v1/search?q={mood}&key=LIVDSRZULELA&limit=1"
        r = requests.get(url)
        return r.json()['results'][0]['media'][0]['gif']['url']
    except:
        return "https://media.tenor.com/images/0f1b67f2f74bba6d553332b26276a707/tenor.gif"

# ✅ STEP 8: Main Logic
def analyze_emotion(language, audio_path):
    if not audio_path:
        return "❌ کوئی آڈیو موصول نہیں ہوئی" if language == "urdu" else "❌ No audio file provided", None

    try:
        result = asr_model.transcribe(audio_path, language="ur" if language == "urdu" else "en")
        text = result.get("text", "").strip()
        if not text:
            return "❌ آڈیو کو سمجھنے میں ناکامی" if language == "urdu" else "❌ Could not transcribe audio", None

        # Emotion detection prompt
        emotion_list = ["خوشی", "اداسی", "غصہ", "تھکاوٹ", "جذباتی", "پر سکون"] if language == "urdu" else \
                       ["happy", "sad", "angry", "tired", "exhausted", "nostalgic", "motivated", "demotivated", "excited", "calm", "anxious"]

        prompt = (
            f"مندرجہ ذیل عبارت کے مطابق جذبات کا اندازہ لگائیں۔ ان میں سے ایک کا انتخاب کریں: {emotion_list}.\n"
            f"عبارت: \"{text}\"\n"
            f"جواب میں لکھیں:\n- Emotion:\n- Reason:"
        ) if language == "urdu" else (
            f"From the following text, detect the most likely human emotion expressed.\n"
            f"Choose one emotion from: {emotion_list}.\n"
            f"Text: \"{text}\"\n"
            f"Respond with:\n- Emotion:\n- Reason:"
        )

        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": prompt}]
        )
        emotion_analysis = response.choices[0].message.content.strip()
        emotion_line = [line for line in emotion_analysis.splitlines() if "Emotion" in line]
        if not emotion_line:
            return "❌ جذبہ شناخت نہ ہو سکا" if language == "urdu" else "❌ Could not detect emotion", None

        emotion = emotion_line[0].split(":")[1].strip()
        playlist = get_playlist(emotion, language)
        poem = get_poem(emotion, language)
        gif_url = search_tenor_gif(emotion)

        playlist_md = "\n".join(playlist) if playlist else "⚠️ کوئی گانا نہیں ملا" if language == "urdu" else "⚠️ No songs found."

        # ✅ STEP 9: Return Beautifully Formatted Output
        if language == "urdu":
            output_text = f"""
<div dir="rtl" style="text-align: right; font-family: 'Noto Nastaliq Urdu', serif; font-size: 16px">

### 📋 متن
{text}

---

### 🎭 جذباتی تجزیہ
{emotion_analysis}

---

### 🎧 گانے (جذبہ: {emotion})
{playlist_md}

---

### 📜 نظم
{poem}

</div>
"""
        else:
            output_text = f"""
### 📝 Transcription
{text}

---

### 🎭 Emotion Analysis
{emotion_analysis}

---

### 🎧 Playlist for *{emotion}*
{playlist_md}

---

### 📜 Poem
{poem}
"""
        return output_text, gif_url

    except Exception as e:
        return f"❌ خرابی: {str(e)}" if language == "urdu" else f"❌ Error: {str(e)}", None

# ✅ STEP 10: Gradio App
gr.Interface(
    fn=analyze_emotion,
    inputs=[
        gr.Radio(["english", "urdu"], label="🌐 Language / زبان منتخب کریں"),
        gr.Audio(type="filepath", label="🎤 Upload or Record Voice")
    ],
    outputs=[
        gr.Markdown(label="📋 Output"),
        gr.Image(label="🖼️ Mood GIF")
    ],
    title="🎙️ Voice-to-Emotion: Pakistani Songs + Urdu Poetry",
    description="🎧 Upload voice → Transcribe → Detect emotion → Get a themed playlist, AI poem, and GIF 🎭"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c44774493c710b7d14.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
